# Experiment 002: Grid-Based Baseline (zaburo kernel)

Implementing the grid placement algorithm that scores 88.33 vs our greedy 164.09.

Key approach:
- Alternating rows with 0/180 degree rotations
- Offset rows for interlocking
- Try different row configurations to find optimal

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

# Set precision for Decimal
getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    """Calculate score (S^2 / N) for a configuration."""
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
def find_best_grid_trees(n: int) -> tuple:
    """
    Grid placement algorithm from zaburo kernel.
    Tries different row configurations to find optimal packing.
    """
    best_score = float("inf")
    best_trees = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.7') / 2
                y = Decimal(r // 2) * Decimal('1.0') if r % 2 == 0 else (Decimal('0.8') + Decimal((r - 1) // 2) * Decimal('1.0'))
                
                row_trees = [
                    ChristmasTree(
                        center_x=Decimal('0.7') * Decimal(i) + x_offset,
                        center_y=y,
                        angle=angle
                    )
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
                
            # Calculate bounding box
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in all_trees])
            min_x, min_y = xys.min(axis=0)
            max_x, max_y = xys.max(axis=0)
            
            side = max(max_x - min_x, max_y - min_y)
            score = side ** 2
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid placement function defined")

Grid placement function defined


In [5]:
# Fix direction functions (rotation tightening)
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        all_points.extend([(float(c[0]) / float(scale_factor), float(c[1]) / float(scale_factor)) for c in coords])
    points_np = np.array(all_points)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x * scale_factor), float(t.center_y * scale_factor)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center

    rotated_trees = []
    for i in range(len(trees)):
        new_x = Decimal(str(rotated[i, 0])) / scale_factor
        new_y = Decimal(str(rotated[i, 1])) / scale_factor
        new_angle = trees[i].angle + Decimal(str(angle_deg))
        new_tree = ChristmasTree(new_x, new_y, new_angle)
        rotated_trees.append(new_tree)
    return rotated_trees

print("Fix direction functions defined")

Fix direction functions defined


In [6]:
# Build all configurations using grid placement
all_configurations = {}
all_scores = {}

print("Building grid configurations...")
for n in range(1, 201):
    score, trees = find_best_grid_trees(n)
    
    if trees is None:
        print(f"WARNING: No valid configuration found for N={n}")
        continue
    
    # Apply fix_direction rotation tightening
    best_side, best_angle = optimize_rotation(trees)
    if abs(best_angle) > 1e-9:
        trees = apply_rotation(trees, best_angle)
    
    all_configurations[n] = [t.clone() for t in trees]
    actual_score = get_score(trees, n)
    all_scores[n] = actual_score
    
    if n % 20 == 0:
        print(f"N={n:3d}: side={float(best_side):.6f}, score={actual_score:.6f}")

print("\nAll configurations built!")

Building grid configurations...


N= 20: side=3.150000, score=0.496125


N= 40: side=4.200000, score=0.441000


N= 60: side=5.200000, score=0.450667


N= 80: side=5.950000, score=0.442531


...

N=120: side=7.000000, score=0.408333


N=140: side=7.350000, score=0.385875


N=160: side=8.050000, score=0.405016


N=180: side=8.400000, score=0.392000


N=200: side=9.000000, score=0.405000

All configurations built!


In [7]:
# Calculate total score
total_score = sum(all_scores.values())
print(f"Total Score: {total_score:.14f}")

# Validate for overlaps
print("\nValidating for overlaps...")
failed_overlap_n = []
for n in range(1, 201):
    if n not in all_configurations:
        continue
    trees = all_configurations[n]
    if has_overlap(trees):
        failed_overlap_n.append(n)
        print(f"  ❌ N={n:03d}: OVERLAP DETECTED!")

if failed_overlap_n:
    print(f"\n❌ Validation FAILED: Overlaps found in N: {failed_overlap_n}")
else:
    print("\n✅ Validation SUCCESSFUL: No overlaps detected.")

print(f"\nImprovement over greedy baseline: {164.09 - total_score:.2f} points")

Total Score: 87.51357685644521

Validating for overlaps...
  ❌ N=003: OVERLAP DETECTED!
  ❌ N=004: OVERLAP DETECTED!
  ❌ N=005: OVERLAP DETECTED!
  ❌ N=031: OVERLAP DETECTED!


  ❌ N=109: OVERLAP DETECTED!


  ❌ N=193: OVERLAP DETECTED!



❌ Validation FAILED: Overlaps found in N: [3, 4, 5, 31, 109, 193]

Improvement over greedy baseline: 76.58 points


In [8]:
# Create submission DataFrame
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in all_configurations[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Round to 6 decimal places
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Prepend 's' to ensure string format
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))
print(f"\nTotal rows: {len(submission)}")

                x           y          deg
id                                        
001_0   s0.212132   s0.087868        s45.0
002_0        s0.0        s0.0         s0.0
002_1       s0.35        s0.8       s180.0
003_0   s0.551246  s-0.091935   s63.434948
003_1   s0.864296   s0.534164   s63.434948
003_2  s-0.007771   s0.578885  s243.434948
004_0   s0.124855  s-0.124889   s15.376252
004_1   s0.799799    s0.06072   s15.376252
004_2   s0.250201    s0.73928  s195.376252
004_3   s0.925145   s0.924889  s195.376252
005_0   s0.451052  s-0.154021   s28.842659
005_1   s1.064216   s0.183663   s28.842659
005_2   s0.371709   s0.715579  s208.842659
005_3   s0.984873   s1.053263  s208.842659
005_4  s-0.031354   s0.721926   s28.842659
006_0        s0.0        s0.0         s0.0
006_1        s0.7        s0.0         s0.0
006_2       s0.35        s0.8       s180.0
006_3       s1.05        s0.8       s180.0
006_4        s0.0        s1.0         s0.0

Total rows: 20100


In [9]:
# Save submission
submission.to_csv('/home/code/experiments/002_grid_baseline/submission.csv')
print("Grid baseline submission saved!")
print(f"\nGrid Baseline Total Score: {total_score:.14f}")

Grid baseline submission saved!

Grid Baseline Total Score: 87.51357685644521


In [ ]:
# Fix overlapping configurations by using a fallback approach
# For small N (3,4,5), use simple manual placements
# For larger N with overlaps, adjust the grid parameters

import random
random.seed(42)

def generate_weighted_angle():
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def greedy_placement(num_trees):
    """Fallback greedy placement for problematic N values."""
    if num_trees == 0:
        return []
    
    placed_trees = [ChristmasTree(angle=random.uniform(0, 360))]
    
    for _ in range(num_trees - 1):
        tree_to_place = ChristmasTree(angle=random.uniform(0, 360))
        placed_polygons = [p.polygon for p in placed_trees]
        tree_index = STRtree(placed_polygons)
        
        best_px, best_py = Decimal('0'), Decimal('0')
        min_radius = Decimal('Infinity')
        
        for _ in range(20):  # More attempts for better placement
            angle = generate_weighted_angle()
            vx = Decimal(str(math.cos(angle)))
            vy = Decimal(str(math.sin(angle)))
            
            radius = Decimal('20.0')
            step_in = Decimal('0.3')
            
            collision_found = False
            while radius >= 0:
                px = radius * vx
                py = radius * vy
                
                candidate_poly = affinity.translate(
                    tree_to_place.polygon,
                    xoff=float(px * scale_factor),
                    yoff=float(py * scale_factor)
                )
                
                possible_indices = tree_index.query(candidate_poly)
                if any((candidate_poly.intersects(placed_polygons[i]) and not
                        candidate_poly.touches(placed_polygons[i]))
                       for i in possible_indices):
                    collision_found = True
                    break
                radius -= step_in
            
            if collision_found:
                step_out = Decimal('0.02')
                while True:
                    radius += step_out
                    px = radius * vx
                    py = radius * vy
                    
                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor)
                    )
                    
                    possible_indices = tree_index.query(candidate_poly)
                    if not any((candidate_poly.intersects(placed_polygons[i]) and not
                               candidate_poly.touches(placed_polygons[i]))
                               for i in possible_indices):
                        break
            else:
                radius = Decimal('0')
                px = Decimal('0')
                py = Decimal('0')
            
            if radius < min_radius:
                min_radius = radius
                best_px = px
                best_py = py
        
        tree_to_place.center_x = best_px
        tree_to_place.center_y = best_py
        tree_to_place.polygon = affinity.translate(
            tree_to_place.polygon,
            xoff=float(tree_to_place.center_x * scale_factor),
            yoff=float(tree_to_place.center_y * scale_factor),
        )
        placed_trees.append(tree_to_place)
    
    return placed_trees

# Fix the overlapping configurations
print("Fixing overlapping configurations...")
for n in failed_overlap_n:
    print(f"  Fixing N={n}...")
    # Use greedy placement for problematic N
    trees = greedy_placement(n)
    
    # Apply rotation tightening
    best_side, best_angle = optimize_rotation(trees)
    if abs(best_angle) > 1e-9:
        trees = apply_rotation(trees, best_angle)
    
    all_configurations[n] = [t.clone() for t in trees]
    all_scores[n] = get_score(trees, n)
    print(f"    N={n}: score={all_scores[n]:.6f}, overlap={has_overlap(trees)}")

print("\nRecalculating total score...")
total_score = sum(all_scores.values())
print(f"Total Score after fixes: {total_score:.14f}")